In [29]:
import mysql.connector
import pandas as pd
import numpy as np
import re

In [30]:
mydb = mysql.connector.connect(
    host="localhost",
    user="student",
    password="student")
print(mydb)

In [31]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS proyecto_final")
mycursor.execute("USE proyecto_final")

In [32]:
camposDict = {'arte_diseno_arquitectura':1, 'agricultura_forestal':2, 'empresariales_administracion':3,
             'economicas':4, 'educacion':5, 'ingenieria_tecnologia':6, 'historia_filosofia_teologia':7,
             'leyes_abogacia':8, 'ciencias_salud':9, 'ciencias_naturales': 10, 'ciencias_sociales':11, 'otros':12}

In [33]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Campos (\
                id int PRIMARY KEY, \
                nombre varchar(255))")
sqlCampos = ("INSERT INTO Campos (id, nombre) VALUES (%s, %s)")
val = []
for k, v in camposDict.items():
    val.append((v, k))
mycursor.executemany(sqlCampos, val)
mydb.commit()

In [34]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Universidades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                url varchar(255), \
                tipo_centro varchar(255))")

In [35]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Facultades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                id_univ int, \
                id_campo int, \
                FOREIGN KEY (id_univ) REFERENCES Universidades(id) ON DELETE CASCADE ON UPDATE CASCADE, \
                FOREIGN KEY (id_campo) REFERENCES Campos(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [36]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Grados (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                id_fac int, \
                nombre varchar(255), \
                presencial bool, \
                nota_corte double, \
                idioma bool, \
                nota_corte_ant double, \
                nota_admision double, \
                rendimiento double, \
                creditos double, \
                precio_creditos double, \
                empleabilidad double, \
                FOREIGN KEY (id_fac) REFERENCES Facultades(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [ ]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Rankings (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                id_univ int, \
                id_campo int, \
                puntuacion int, \
                FOREIGN KEY (id_univ) REFERENCES Universidades(id) ON DELETE CASCADE ON UPDATE CASCADE, \
                FOREIGN KEY (id_campo) REFERENCES Campos(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [37]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('Campos',)
('Facultades',)
('Grados',)
('Universidades',)


In [38]:
df = pd.read_csv("file:/home/student/Downloads/Datos_QEDU.csv").replace(to_replace='\(.+\)', value = '', regex = True)\
.drop_duplicates(subset=['NOMBRE_UNIV', 'NOMBRE_CENTRO', 'NOM_TITULA']).reset_index()

In [39]:
df2 = df.replace(['Pública / Propio', 'Pública / Adscrito**'], 'Pública').replace(['Privada / Propio', 'Privada / Adscrito**'], 'Privada').fillna(np.nan).replace([np.nan], [None])
dfUniv = df2.groupby(['NOMBRE_UNIV', 'URL_INFO', 'TIPO_UNIV_CENTRO'], dropna=False).size().reset_index().iloc[:, :-1]
dfUniv

,NOMBRE_UNIV,URL_INFO,TIPO_UNIV_CENTRO
0,Atlántico Medio,https://www.universidadatlanticomedio.es/plane...,Privada
1,CUNEF Universidad,https://www.cunef.edu/estudios/cunef-grado/,Privada
2,ESIC Universidad,https://www.esic.edu/university/grados,Privada
3,Hesperides,https://hesperides.edu.es/estudios/,Privada
4,IE Universidad,http://www.ie.edu/es/universidad/estudios/ofer...,Privada
...,...,...,...
83,Universidad de la Rioja,https://www.unirioja.es/estudios/grados/,Pública
84,Universidad de las Illes Balears,http://estudis.uib.cat/es/grau/,Pública
85,Universidad del País Vasco/Euskal Herriko Unib...,https://www.ehu.eus/es/web/estudiosdegrado-gra...,Pública
86,Universitat Abat Oliba CEU,https://www.uaoceu.es/estudios?tipos_estudio=3...,Privada


In [40]:
sqlUniv = "INSERT INTO Universidades (nombre, url, tipo_centro) VALUES (%s, %s, %s)"
for index, row in dfUniv.iterrows():
    val = (row['NOMBRE_UNIV'], row['URL_INFO'], row['TIPO_UNIV_CENTRO'])
    mycursor.execute(sqlUniv, val)
mydb.commit()

In [41]:
dfFac = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO'], dropna=False).size().reset_index().iloc[:, :-1]
dfFac

,NOMBRE_UNIV,NOMBRE_CENTRO
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas
1,CUNEF Universidad,Escuela Politécnica Superior
2,CUNEF Universidad,"Facultad de Empresa, Economía y Derecho"
3,ESIC Universidad,Facultad de Ciencias Jurídicas y Sociales
4,Hesperides,Escuela de Grados
...,...,...
1062,Universitat de València,Facultad de Psicología y Logopedia
1063,Universitat de València,Facultad de Química
1064,Universitat de València,Facultat d'Infermeria i Podologia
1065,Universitat de València,Facultat de Fisioteràpia


In [42]:
campos = []
arte_diseno_arquitectura = ['Arquitectura', 'Diseño', 'Arte']
agricultura_forestal = ['Agricultura', 'Forestal']
empresariales_administracion = ['Empresa', 'Empresariales', 'Relaciones', 'Marketing', 'Comercio']
economicas = ['Económicas', 'Finan']
educacion = ['Educación', 'Enseñanza', 'Magisteri', 'Profesorado']
ingenieria_tecnologia = ['Ingeniería', 'Politécnica', 'Informática', 'Náutica', 'Ingeniero', 'Tecnología']
historia_filosofia_teologia = ['Filosofía', 'Historia']
leyes_abogacia = ['Derecho', 'Jurídicas']
ciencias_salud = ['Psicología', 'Salud', 'Medicina', 'Farmacia', 'Biol', 'Óptica', 'Sociosanitarias',
                  'Alimentación', 'Deporte', 'Odontología', 'Enfermería', 'Fisio', 'Infermeria']
ciencias_sociales = ['Comunicación', 'Humanidades', 'Sociología', 'Humana', 'Turismo', 'Criminología', 'Letras',
                     'Social', 'Filología', 'Lengua', 'Traducción', 'Información' 'Documentación', 'Política']
ciencias_naturales = ['Física', 'Matemáticas', 'Veterinaria', 'Naturaleza', 'Ciencias', 'Bioq',
                      'Geolo', 'Química', 'Estadísti']
for index, row in dfFac.iterrows():
    facultad = row['NOMBRE_CENTRO']
    if any([x in facultad for x in arte_diseno_arquitectura]):
        campos.append('arte_diseno_arquitectura')
    elif any([x in facultad for x in agricultura_forestal]):
        campos.append('agricultura_forestal')
    elif any([x in facultad for x in empresariales_administracion]):
        campos.append('empresariales_administracion')
    elif any([x in facultad for x in economicas]):
        campos.append('economicas')
    elif any([x in facultad for x in educacion]):
        campos.append('educacion')
    elif any([x in facultad for x in ingenieria_tecnologia]):
        campos.append('ingenieria_tecnologia')
    elif any([x in facultad for x in historia_filosofia_teologia]):
        campos.append('historia_filosofia_teologia')
    elif any([x in facultad for x in leyes_abogacia]):
        campos.append('leyes_abogacia')
    elif any([x in facultad for x in ciencias_salud]):
        campos.append('ciencias_salud')
    elif any([x in facultad for x in ciencias_naturales]):
        campos.append('ciencias_naturales')
    elif any([x in facultad for x in ciencias_sociales]):
        campos.append('ciencias_sociales')
    else:
        campos.append('otros')
dfFac2 = dfFac.assign(CAMPOS=campos)

In [43]:
sqlFac = "INSERT INTO Facultades (nombre, id_univ, id_campo) VALUES (%s, %s, %s)"
sqlQueryUniv = "SELECT id FROM Universidades WHERE nombre = %s"
uniAnterior = ""
idUni = -1
for index, row in dfFac2.iterrows():
    uniActual = row['NOMBRE_UNIV']
    if uniAnterior != uniActual:
        mycursor.execute(sqlQueryUniv, (uniActual, ))
        myresult = mycursor.fetchall()
        idUni = myresult[0][0]
        uniAnterior = uniActual
    val = (row['NOMBRE_CENTRO'], idUni, camposDict.get(row['CAMPOS']))
    mycursor.execute(sqlFac, val)
mydb.commit()

In [44]:
dfGrad = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO', 'NOM_TITULA', 'PRESENCIALIDAD', 'NOTA_CORTE', 'IDIOMAEXTRANJERO',
                      'NOTA_CORTE_ANT','NOTA_ADMISION', 'RENDIMIENTO',
                      'CREDITOS', 'PRECIO', 'TASA_AFIL'], dropna=False).size().reset_index().iloc[:, :-1]
dfNone= dfGrad.fillna(np.nan).replace([np.nan], [None])
dfNone

,NOMBRE_UNIV,NOMBRE_CENTRO,NOM_TITULA,PRESENCIALIDAD,NOTA_CORTE,IDIOMAEXTRANJERO,NOTA_CORTE_ANT,NOTA_ADMISION,RENDIMIENTO,CREDITOS,PRECIO,TASA_AFIL
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Administración y Dirección de Empresas,Sí,None,No,None,None,87.0,240.0,None,None
1,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Cine,Sí,None,No,None,None,None,240.0,None,None
2,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Comunicación,Sí,None,No,None,None,94.0,240.0,None,None
3,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Derecho,Varias modalidades*,None,No,None,None,23.0,240.0,None,None
4,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Maestro en Educación Infantil,Varias modalidades*,None,No,None,None,96.0,240.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3565,Universitat de València,Facultat d'Infermeria i Podologia,Grado en Enfermería,Sí,12.35,No,12.51,None,90.0,240.0,18.0,78.3
3566,Universitat de València,Facultat d'Infermeria i Podologia,Grado en Podología,Sí,11.03,No,11.08,11.08,87.0,240.0,18.0,80.7
3567,Universitat de València,Facultat de Fisioteràpia,Grado en Fisioterapia,Sí,11.87,No,12.05,None,91.0,240.0,18.0,76.4
3568,Universitat de València,Facultat de Magisteri,Grado en Maestro/a en Educación Infantil,Sí,9.59,No,9.75,10.37,96.0,240.0,12.79,77.1


In [45]:
sqlGrad = "INSERT INTO Grados (id_fac, nombre, presencial, nota_corte, idioma, \
                                    nota_corte_ant, nota_admision, rendimiento, creditos, precio_creditos, \
                                    empleabilidad) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sqlQueryFac = "SELECT id FROM Facultades WHERE id_univ = (SELECT id FROM Universidades WHERE nombre = %s) AND nombre = %s"
uniAnterior = ""
facAnterior = ""
idFac = -1
for index, row in dfNone.iterrows():
    uniActual = row['NOMBRE_UNIV']
    facActual = row['NOMBRE_CENTRO']
    if uniAnterior != uniActual or facAnterior != facActual:
        mycursor.execute(sqlQueryFac, (uniActual, facActual))
        myresult = mycursor.fetchall()
        idFac = myresult[0][0]
        uniAnterior = uniActual
        facAnterior = facActual
    if row['PRESENCIALIDAD']=="No": 
        presencialidad=0 
    else: 
        presencialidad=1
    if row['IDIOMAEXTRANJERO']=="No":
        idioma=0 
    else: 
        idioma=1
    val = (idFac, row['NOM_TITULA'], presencialidad, row['NOTA_CORTE'], 
           idioma, row['NOTA_CORTE_ANT'], row['NOTA_ADMISION'], row['RENDIMIENTO'],
           row['CREDITOS'], row['PRECIO'], row['TASA_AFIL'])
    mycursor.execute(sqlGrad, val)
mydb.commit()

[(1, 'arte_diseno_arquitectura'),
 (2, 'agricultura_forestal'),
 (3, 'empresariales_administracion'),
 (4, 'economicas'),
 (5, 'educacion'),
 (6, 'ingenieria_teconologia'),
 (7, 'historia_filosofia_teologia'),
 (8, 'leyes_abogacia'),
 (9, 'ciencias_salud'),
 (10, 'ciencias_naturales'),
 (11, 'ciencias_sociales'),
 (12, 'otros')]

,NOMBRE_UNIV,NOMBRE_CENTRO
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas
1,CUNEF Universidad,Escuela Politécnica Superior
2,CUNEF Universidad,"Facultad de Empresa, Economía y Derecho"
3,ESIC Universidad,Facultad de Ciencias Jurídicas y Sociales
4,Hesperides,Escuela de Grados
...,...,...
1062,Universitat de València,Facultad de Psicología y Logopedia
1063,Universitat de València,Facultad de Química
1064,Universitat de València,Facultat d'Infermeria i Podologia
1065,Universitat de València,Facultat de Fisioteràpia


,NOMBRE_UNIV,NOMBRE_CENTRO,CAMPOS
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,leyes_abogacia
1,CUNEF Universidad,Escuela Politécnica Superior,ingenieria_tecnologia
2,CUNEF Universidad,"Facultad de Empresa, Economía y Derecho",empresariales_administracion
3,ESIC Universidad,Facultad de Ciencias Jurídicas y Sociales,leyes_abogacia
4,Hesperides,Escuela de Grados,otros
...,...,...,...
1062,Universitat de València,Facultad de Psicología y Logopedia,ciencias_salud
1063,Universitat de València,Facultad de Química,ciencias_naturales
1064,Universitat de València,Facultat d'Infermeria i Podologia,ciencias_salud
1065,Universitat de València,Facultat de Fisioteràpia,ciencias_salud
